# LoC image analysis

This notebook is designed to take the raw output of a LoC data set, monolayer or bilayer, and:
 (trying to check if dask multichan works)

5. Localise and measure properties
6. Unite the localisations over the z-range
7. Save out the Z-tracks
8. Extract the maximum intensity from each cell

In [17]:
import os
import glob
from octopusheavy import DaskOctopusHeavyLoader
import napari
from skimage.io import imshow,  imsave, imread
import napari
import btrack
from tqdm.auto import tqdm
import dask.array as da

# Loading images

Define root path and individual experiment IDs

In [18]:
root_path = '/run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/shared/Lung on chip/Light microscopy'

In [19]:
expt_IDs = ['co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-SPC-PDPN-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60_/images/',
            'co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-VWF-iCAM1-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_70_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-AQP5-proSPC-ZO1/Day7_static/_20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_150_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-AQP5-proSPC-ZO1/Day14_static/_20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_262_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-CAV1-proSPC-ZO1/Day7_static/_20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_130_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-CAV1-proSPC-ZO1/Day14_static/_20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_253_/images/'
           ]

## Check to see what channels each expt has

In [20]:
for i, expt in enumerate(expt_IDs):
    images = DaskOctopusHeavyLoader(os.path.join(root_path, expt), remove_background=False)
    if 'MASK1' in [channel.name for channel in images.channels]:
        images = DaskOctopusHeavyLoader(os.path.join(root_path, expt), remove_background=False)
        print(i, images.channels)
        dirname = os.path.dirname(images.files('CH1')[0]).replace('images','')
        tracks_fn = os.path.join(dirname, 'tracks.h5')
        tracks_fn = tracks_fn.split('Light microscopy/')[-1].replace('/', '_')
#         tracks_fn = os.path.join(root_path, expt.replace('images/', 'tracks.hdf5'))
        print('Tracks exist:', os.path.exists(tracks_fn))

0 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
Tracks exist: True
1 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
Tracks exist: True
2 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
Tracks exist: True
3 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
Tracks exist: True
4 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
Tracks exist: True
5 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
Tracks exist: True


In [38]:
tracks_fn = 'co-culture_iVECs+iAT2AT1_Folder_20220808_A2-A5_analysis_20221125_DAPI-SPC-PDPN-ZO1__20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60__tracks.csv'

In [39]:
expt = expt_IDs[0]
images = DaskOctopusHeavyLoader(os.path.join(root_path, expt), remove_background=False)
dirname = os.path.dirname(images.files('CH1')[0]).replace('images','')
tracks_fn = os.path.join(dirname, 'tracks.h5')
tracks_fn = tracks_fn.split('Light microscopy/')[-1].replace('/', '_')
with btrack.dataio.HDF5FileHandler(tracks_fn, "r", obj_type="obj_type_1") as hdf:
        tracks = hdf.tracks
#         tracks = [track for track in tracks if len(track) > 7]
        data, new_properties, new_graph = btrack.utils.tracks_to_napari(tracks, ndim=2)
print('Loaded expt:', expt)

[INFO][2022/12/02 02:57:31 PM] Opening HDF file: co-culture_iVECs+iAT2AT1_Folder_20220808_A2-A5_analysis_20221125_DAPI-SPC-PDPN-ZO1__20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60__tracks.h5...
[INFO][2022/12/02 02:57:31 PM] Loading tracks/obj_type_1
[INFO][2022/12/02 02:57:31 PM] Loading LBEP/obj_type_1
[INFO][2022/12/02 02:57:31 PM] Loading objects/obj_type_1 (59338, 5) (59338 filtered: None)
[INFO][2022/12/02 02:57:32 PM] Closing HDF file: co-culture_iVECs+iAT2AT1_Folder_20220808_A2-A5_analysis_20221125_DAPI-SPC-PDPN-ZO1__20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60__tracks.h5


NameError: free variable 'dummy_obj' referenced before assignment in enclosing scope

In [9]:
tracks[0]

NameError: name 'tracks' is not defined

## Checking tracks prior to plotting single cell info

In [ ]:
import napari

In [ ]:
viewer = napari.Viewer()

viewer.add_image(ch0, colormap='blue', blending = 'additive')
viewer.add_image(ch3, colormap= 'gray', blending = 'additive')
viewer.add_image(ch1, colormap='red', blending = 'additive')
viewer.add_image(ch2, colormap= 'green', blending = 'additive')

viewer.add_labels(mask_stack)

viewer.add_tracks(ch1_data)
viewer.add_tracks(ch2_data)
